
# Import Data

Importing AXP data.

In [8]:

import pandas as pd

amex_df = pd.read_csv('../../data/gen/usAXP_df.csv',
                           parse_dates=True,
                           index_col=0)
# Strip whitespace from column names
amex_df.columns = amex_df.columns.str.strip()
# Print
amex_df.head()


,close,high,low,p_change,open,pre_close,volume,date,date_week,atr21,atr14,key
,,,,,,,,,,,,
2017-06-12,80.17,80.73,79.95,-0.174,80.35,80.31,3352279,20170612,0,0.780000,0.780000,0
2017-06-13,80.59,80.74,80.07,0.524,80.20,80.17,3174361,20170613,1,0.722381,0.721071,1
2017-06-14,80.84,80.92,79.62,0.310,80.11,80.59,4013089,20170614,2,0.933535,0.942224,2
2017-06-15,80.70,81.24,80.23,-0.173,80.38,80.84,2773369,20170615,3,0.955464,0.962959,3
2017-06-16,81.45,81.48,80.77,0.929,80.86,80.70,5914676,20170616,4,0.896598,0.896962,4



# Even Faster Approach: Using Numba

Optimize Calculation using Numba

## Processing Data

Adding slope & velocity to each day.


In [9]:

import numpy as np

extra_df = amex_df.copy()
extra_df['slope'] = pd.Series(np.gradient(extra_df.close), extra_df.index, name='slope')
extra_df['velocity'] = pd.Series(np.gradient(extra_df.slope), extra_df.index, name='velocity')
# Print
extra_df.head()


,close,high,low,p_change,open,pre_close,volume,date,date_week,atr21,atr14,key,slope,velocity
,,,,,,,,,,,,,,
2017-06-12,80.17,80.73,79.95,-0.174,80.35,80.31,3352279,20170612,0,0.780000,0.780000,0,0.420,-0.0850
2017-06-13,80.59,80.74,80.07,0.524,80.20,80.17,3174361,20170613,1,0.722381,0.721071,1,0.335,-0.1825
2017-06-14,80.84,80.92,79.62,0.310,80.11,80.59,4013089,20170614,2,0.933535,0.942224,2,0.055,-0.0150
2017-06-15,80.70,81.24,80.23,-0.173,80.38,80.84,2773369,20170615,3,0.955464,0.962959,3,0.305,0.2675
2017-06-16,81.45,81.48,80.77,0.929,80.86,80.70,5914676,20170616,4,0.896598,0.896962,4,0.590,0.1125



## Performing Backtests

1. Prepare Parameters
1. Dfine a DataFrame to handle results
1. For Loops
1. Prepare Maps & Filters

## Preparing Parameters


In [10]:

import itertools

buy_slope_threshold_list = np.arange(-0.5, 0.5, 0.1)
buy_velocity_threshold_list = np.arange(-0.5, 0.5, 0.1)
close_slope_threshold_list = np.arange(-0.2, 0.2, 0.1)
close_velocity_threshold_list = np.arange(-0.2, 0.2, 0.1)
sell_slope_threshold_list = np.arange(-0.5, 0.5, 0.1)
sell_velocity_threshold_list = np.arange(-0.5, 0.5, 0.1)

task_list = list(itertools.product(
    buy_slope_threshold_list, 
    buy_velocity_threshold_list, 
    close_slope_threshold_list, 
    close_velocity_threshold_list, 
    sell_slope_threshold_list, 
    sell_velocity_threshold_list
))
print("Params Ready, {} tasks to run. ".format(len(task_list)))


Params Ready, 160000 tasks to run. 



## For Loops


In [11]:

import concurrent.futures
from concurrent.futures.process import ProcessPoolExecutor
from datetime import datetime
from tqdm import tqdm

def single_test(data_df, 
                buy_slope_threshold, 
                buy_velocity_threshold, 
                close_slope_threshold, 
                close_velocity_threshold, 
                sell_slope_threshold, 
                sell_velocity_threshold):
        
        # Prepare Maps & Filters 
        
        long_mask = (data_df.slope > buy_slope_threshold) & (data_df.velocity > buy_velocity_threshold)
        long_close = (data_df.slope < -close_slope_threshold) & (data_df.velocity < -close_velocity_threshold)
        long_series = data_df.p_change[long_mask]
        long_array = long_series.values
        long_profit = np.product(long_array/100 + 1) - 1
        
        short_mask = (data_df.slope < sell_slope_threshold) & (data_df.velocity < sell_velocity_threshold)
        short_close = (data_df.slope > close_slope_threshold) & (data_df.velocity > close_velocity_threshold)
        short_series = data_df.p_change[short_mask]
        short_array = short_series.values
        short_profit = np.abs(np.product(short_array/100 - 1)) - 1
    
        trade_profit = long_profit + short_profit
    
        test_result = [buy_slope_threshold, 
                       buy_velocity_threshold, 
                       close_slope_threshold, 
                       close_velocity_threshold, 
                       sell_slope_threshold, 
                       sell_velocity_threshold, 
                       trade_profit * 100]
        return test_result

start_time = datetime.now()

with ProcessPoolExecutor() as executor:
    """
    By default, ProcessPool uses maximum available number of cores to process.
    """
    
    """
    Equivalent to executor.map(fn, *iterables),
    but displays a tqdm-based progress bar.
    
    Does not support timeout or chunksize as executor.submit is used internally
    
    **kwargs is passed to tqdm.
    """
    futures_list = []
    kwargs = {
        'total': len(futures_list),
        'unit': 'tests',
        'unit_scale': True,
        'leave': True
    }
        
    futures_list = [executor.submit(single_test, 
                                    extra_df,
                                    buy_slope_threshold, 
                                    buy_velocity_threshold, 
                                    close_slope_threshold, 
                                    close_velocity_threshold, 
                                    sell_slope_threshold, 
                                    sell_velocity_threshold) for buy_slope_threshold, \
                                                                 buy_velocity_threshold, \
                                                                 close_slope_threshold, \
                                                                 close_velocity_threshold, \
                                                                 sell_slope_threshold, \
                                                                 sell_velocity_threshold in task_list]
    print("Starting {} tasks".format(len(futures_list)))
    
    for f in tqdm(concurrent.futures.as_completed(futures_list), **kwargs):
        # done_callback runs on the main process
        result_row = f.result()
        
        final_result = final_result.append(pd.Series(result_row, index=final_result.columns), ignore_index=True)

    print("Finished {} tasks in {}".format(len(task_list), datetime.now() - start_time))


1.00ktests [00:01, 503tests/s]


Starting 1000 tasks
Finished 160000 tasks in 0:00:02.059098



# Analysing Results


In [7]:
        
final_result = final_result.sort_values(by=['profit_percentage'],
                                        ascending=False)
final_result.head(100)


,buy_slope_threshold,buy_velocity_threshold,close_slope_threshold,close_velocity_threshold,sell_slope_threshold,sell_velocity_threshold,profit_percentage
59,-0.5,-0.5,-0.2,-0.2,-1.110223e-16,4.000000e-01,119.425931
159,-0.5,-0.5,-0.2,-0.2,-1.110223e-16,4.000000e-01,119.425931
149,-0.5,-0.5,-0.2,-0.2,-1.000000e-01,4.000000e-01,119.000359
49,-0.5,-0.5,-0.2,-0.2,-1.000000e-01,4.000000e-01,119.000359
39,-0.5,-0.5,-0.2,-0.2,-2.000000e-01,4.000000e-01,118.755829
139,-0.5,-0.5,-0.2,-0.2,-2.000000e-01,4.000000e-01,118.755829
127,-0.5,-0.5,-0.2,-0.2,-3.000000e-01,4.000000e-01,117.314780
29,-0.5,-0.5,-0.2,-0.2,-3.000000e-01,4.000000e-01,117.314780
69,-0.5,-0.5,-0.2,-0.2,1.000000e-01,4.000000e-01,117.219541
169,-0.5,-0.5,-0.2,-0.2,1.000000e-01,4.000000e-01,117.219541
